In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
df_raw = pd.read_csv("leafly_merged.csv")

In [3]:
df_raw

,strain,species,report,effect,aroma,X..Delta9-THC,X..CBD,X..CBN,X..CBG,X..CBC,...,X..Guaiol,X..CaryophylleneOxide,X..Alpha-Bisabolol,X..Beta-Caryophyllene,X..Alpha-Humulene,X..p-Cymene,X..cis-Nerolidol,X..trans-Nerolidol,X..trans-Ocimene,X..?-Terpinene\t
0,dairy-queen,Hybrid,Migraine pain 6; nausea 4 when I medicated rou...,"['Energetic', 'Euphoric', 'Hungry', 'Relaxed',...",[],8.5,0.01,0.04,0.11,0.10,...,0.04,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dairy-queen,Hybrid,Tasted delicious and great overall well-balanc...,"['Aroused', 'Euphoric', 'Focused', 'Happy', 'R...","['Butter', 'Cheese', 'Spicy/Herbal', 'Vanilla']",8.5,0.01,0.04,0.11,0.10,...,0.04,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dairy-queen,Hybrid,This version was an outdoor grow. Very loose ...,"['Aroused', 'Creative', 'Euphoric', 'Focused',...","['Berry', 'Cheese', 'Sweet', 'Vanilla']",8.5,0.01,0.04,0.11,0.10,...,0.04,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dairy-queen,Hybrid,Dq compares well to hybrids like Super Lemon H...,"['Euphoric', 'Relaxed', 'Uplifted']","['Blue Cheese', 'Cheese', 'Grape', 'Lavender',...",8.5,0.01,0.04,0.11,0.10,...,0.04,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dairy-queen,Hybrid,"Favorite strain that ive tried. Low smell, sma...",[],[],8.5,0.01,0.04,0.11,0.10,...,0.04,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,blackberry-kush,Indica,Loving this for Midnight tokin&#39; and Dayti...,"['Euphoric', 'Happy', 'Relaxed', 'Sleepy']",[],18.4,0.01,0.03,0.87,0.35,...,0.02,0.0,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42973,blackberry-kush,Indica,At my knowledge &quot;Blackberry Kush is SATIV...,[],[],18.4,0.01,0.03,0.87,0.35,...,0.02,0.0,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42974,blackberry-kush,Indica,Chase,"['Euphoric', 'Dry Mouth']",[],18.4,0.01,0.03,0.87,0.35,...,0.02,0.0,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42975,blackberry-kush,Indica,Im 17 now but when i smoked this i felt sooooo...,"['Euphoric', 'Giggly', 'Happy', 'Hungry', 'Rel...",[],18.4,0.01,0.03,0.87,0.35,...,0.02,0.0,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42977 entries, 0 to 42976
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   strain                 42977 non-null  object 
 1   species                42977 non-null  object 
 2   report                 42977 non-null  object 
 3   effect                 42977 non-null  object 
 4   aroma                  42977 non-null  object 
 5   X..Delta9-THC          42977 non-null  float64
 6   X..CBD                 42977 non-null  float64
 7   X..CBN                 42977 non-null  float64
 8   X..CBG                 42977 non-null  float64
 9   X..CBC                 42977 non-null  float64
 10  X..THCV                42977 non-null  float64
 11  X..CBDa                27506 non-null  float64
 12  X..Delta8-THC          25439 non-null  float64
 13  X..CBGa                27506 non-null  float64
 14  X..THCa                25439 non-null  float64
 15  X.

In [5]:
review = df_raw['report']

In [6]:
df_review = pd.DataFrame(review)

In [7]:
df_review

,report
0,Migraine pain 6; nausea 4 when I medicated rou...
1,Tasted delicious and great overall well-balanc...
2,This version was an outdoor grow. Very loose ...
3,Dq compares well to hybrids like Super Lemon H...
4,"Favorite strain that ive tried. Low smell, sma..."
...,...
42972,Loving this for Midnight tokin&#39; and Dayti...
42973,At my knowledge &quot;Blackberry Kush is SATIV...
42974,Chase
42975,Im 17 now but when i smoked this i felt sooooo...


In [8]:
df_ht = pd.read_csv("df_ht.csv")

In [9]:
df_ht

,review,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,dry mouth,...,vanilla,violet,woody,species,X..Delta9-THC,X..CBD,X..D-Limonene,X..Beta-Myrcene,X..Beta-Pinene,X..Linalool
0,migrain pain nausea medic round take bout min ...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
1,tast delici great overal well-balanc effect hi...,0,0,1,0,0,0,0,0,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
2,version outdoor grow loos bud good amount pain...,0,0,1,0,1,0,1,0,1,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
3,compar well hybrid like super lemon haze possi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
4,favorit strain tri low smell small pine start ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,love midnight daytim kind brain massag sweet s...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42973,knowledg quot blackberri domin cross black ras...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42974,chase,0,0,0,0,0,0,0,0,1,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42975,smoke felt good whole high get horni make happ...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02


In [10]:
df_ht[df_ht['review'].isnull()]

,review,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,dry mouth,...,vanilla,violet,woody,species,X..Delta9-THC,X..CBD,X..D-Limonene,X..Beta-Myrcene,X..Beta-Pinene,X..Linalool
430,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.77,0.01,0.67,0.80,0.11,0.00
434,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.77,0.01,0.67,0.80,0.11,0.00
445,NaN,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0.77,0.01,0.67,0.80,0.11,0.00
557,NaN,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0.77,0.01,0.67,0.80,0.11,0.00
560,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.77,0.01,0.67,0.80,0.11,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42585,NaN,0,0,0,0,0,0,0,0,1,...,0,0,0,1,18.40,0.01,0.19,0.35,0.03,0.02
42752,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.40,0.01,0.19,0.35,0.03,0.02
42800,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.40,0.01,0.19,0.35,0.03,0.02
42803,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.40,0.01,0.19,0.35,0.03,0.02


In [11]:
df_ht['review'] = df_ht['review'].fillna('')

In [12]:
df_ht.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42977 entries, 0 to 42976
Data columns (total 86 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review           42977 non-null  object 
 1   anxiety          42977 non-null  int64  
 2   anxious          42977 non-null  int64  
 3   aroused          42977 non-null  int64  
 4   arthritis        42977 non-null  int64  
 5   creative         42977 non-null  int64  
 6   depression       42977 non-null  int64  
 7   dizzy            42977 non-null  int64  
 8   dry eyes         42977 non-null  int64  
 9   dry mouth        42977 non-null  int64  
 10  energetic        42977 non-null  int64  
 11  epilepsy         42977 non-null  int64  
 12  euphoric         42977 non-null  int64  
 13  eye pressure     42977 non-null  int64  
 14  fatigue          42977 non-null  int64  
 15  focused          42977 non-null  int64  
 16  giggly           42977 non-null  int64  
 17  happy       

In [13]:
df_ht2 = df_ht.drop(['review'], axis = 1)

In [14]:
df_ht2

,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,dry mouth,energetic,...,vanilla,violet,woody,species,X..Delta9-THC,X..CBD,X..D-Limonene,X..Beta-Myrcene,X..Beta-Pinene,X..Linalool
0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
2,0,0,1,0,1,0,1,0,1,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42973,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42974,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02


In [15]:
df_no_tfidf = pd.read_csv("df_final_mlp.csv")

In [16]:
df_no_tfidf

,hybrid,indica,sativa,lsa_0,lsa_1,lsa_2,lsa_3,lsa_4,lsa_5,lsa_6,...,X..3-Carene,X..Alpha-Terpinene,X..Ocimene,X..Eucalyptol,X..Terpinolene,X..Isopulegol,X..Geraniol,X..Guaiol,X..CaryophylleneOxide,X..Alpha-Bisabolol
0,1,0,0,0.418311,0.274948,-0.046076,0.079432,-0.283345,0.110858,-0.001165,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
1,1,0,0,0.348428,-0.018824,0.353926,0.053830,0.027603,0.041568,-0.021110,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
2,1,0,0,0.384142,0.220097,-0.067963,0.086681,-0.166829,-0.105413,0.004228,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
3,1,0,0,0.353262,0.104061,0.170056,-0.155247,0.225551,-0.090213,-0.154757,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
4,1,0,0,0.391750,-0.092959,-0.142575,-0.140255,0.217436,-0.074387,-0.211877,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,0,1,0,0.195335,-0.041536,0.022845,-0.037624,0.018596,-0.038189,-0.089364,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263
42973,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263
42974,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263
42975,0,1,0,0.424053,0.201999,-0.126061,0.001543,-0.122060,-0.079580,0.264316,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263


In [17]:
df_no_tfidf = df_no_tfidf.drop(['lsa_0',
 'lsa_1',
 'lsa_2',
 'lsa_3',
 'lsa_4',
 'lsa_5',
 'lsa_6',
 'lsa_7',
 'lsa_8',
 'lsa_9',
 'lsa_10',
 'lsa_11',
 'lsa_12',
 'lsa_13',
 'lsa_14',
 'lsa_15',
 'lsa_16',
 'lsa_17',
 'lsa_18',
 'lsa_19',
 'lsa_20',
 'lsa_21',
 'lsa_22',
 'lsa_23',
 'lsa_24',
 'lsa_25',
 'lsa_26',
 'lsa_27',
 'lsa_28',
 'lsa_29',
 'lsa_30'], axis = 1)

In [18]:
df_no_tfidf

,hybrid,indica,sativa,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,...,X..3-Carene,X..Alpha-Terpinene,X..Ocimene,X..Eucalyptol,X..Terpinolene,X..Isopulegol,X..Geraniol,X..Guaiol,X..CaryophylleneOxide,X..Alpha-Bisabolol
0,1,0,0,0,0,0,0,0,0,1,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
1,1,0,0,0,0,1,0,0,0,0,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
2,1,0,0,0,0,1,0,1,0,1,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
3,1,0,0,0,0,0,0,0,0,0,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
4,1,0,0,0,0,0,0,0,0,0,...,1.0,0.75,0.0,0.0,1.0,0.0,0.0,0.444444,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,0,1,0,0,0,0,0,0,0,0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263
42973,0,1,0,0,0,0,0,0,0,0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263
42974,0,1,0,0,0,0,0,0,0,0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263
42975,0,1,0,0,0,0,0,0,0,0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.105263


In [23]:
df_no_tfidf.columns.to_list()

['hybrid',
 'indica',
 'sativa',
 'anxiety',
 'anxious',
 'aroused',
 'arthritis',
 'creative',
 'depression',
 'dizzy',
 'dry eyes',
 'dry mouth',
 'energetic',
 'epilepsy',
 'euphoric',
 'eye pressure',
 'fatigue',
 'focused',
 'giggly',
 'happy',
 'headache',
 'hungry',
 'migraines',
 'pain',
 'paranoid',
 'relaxed',
 'seizures',
 'sleepy',
 'spasticity',
 'stress',
 'talkative',
 'tingly',
 'uplifted',
 'ammonia',
 'apple',
 'apricot',
 'berry',
 'blue cheese',
 'blueberry',
 'butter',
 'cheese',
 'chemical',
 'chestnut',
 'citrus',
 'coffee',
 'diesel',
 'earthy',
 'flowery',
 'fruit',
 'grape',
 'grapefruit',
 'honey',
 'lavender',
 'lemon',
 'lime',
 'mango',
 'menthol',
 'mint',
 'nutty',
 'orange',
 'peach',
 'pear',
 'pepper',
 'pine',
 'pineapple',
 'plum',
 'pungent',
 'rose',
 'sage',
 'skunk',
 'spicy/herbal',
 'strawberry',
 'sweet',
 'tar',
 'tea',
 'tobacco',
 'tree',
 'tropical',
 'vanilla',
 'violet',
 'woody',
 'X..Delta9-THC',
 'X..CBD',
 'X..CBN',
 'X..CBG',
 'X..

In [15]:
np.linspace(0.01,1,10)

array([0.01, 0.12, 0.23, 0.34, 0.45, 0.56, 0.67, 0.78, 0.89, 1.  ])

In [19]:
#ngram_range = [(1,1), (1,2), (2,2)] #(2,2) started giving the error?
ngram_range = [(1,1), (1,2)]
min_df = [0.01, 0.05, 0.1]
max_df = [0.9, 0.95, 1]
#n_components = [2, 31, 62]

In [20]:
param_list = []
for a in ngram_range:
    for b in min_df:
        for c in max_df:
            #for d in n_components:
            param_list.append([a,b,c])
                #param_list.append([a,b,c,d])

In [21]:
df_param = pd.DataFrame(param_list, columns = ['ngram_range', 'min_df', 'max_df'])
#df_param = pd.DataFrame(param_list, columns = ['ngram_range', 'min_df', 'max_df', 'n_components'])

In [22]:
df_param

,ngram_range,min_df,max_df
0,"(1, 1)",0.01,0.90
1,"(1, 1)",0.01,0.95
2,"(1, 1)",0.01,1.00
3,"(1, 1)",0.05,0.90
4,"(1, 1)",0.05,0.95
5,"(1, 1)",0.05,1.00
6,"(1, 1)",0.10,0.90
7,"(1, 1)",0.10,0.95
8,"(1, 1)",0.10,1.00
9,"(1, 2)",0.01,0.90


In [43]:
df_lsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42977 entries, 0 to 42976
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       42977 non-null  float64
 1   1       42977 non-null  float64
dtypes: float64(2)
memory usage: 671.6 KB


In [61]:
df_lsa

,0,1
0,0.336072,0.169639
1,0.237074,-0.122952
2,0.264813,0.128810
3,0.243727,-0.004353
4,0.242857,0.001288
...,...,...
42972,0.178815,-0.018841
42973,0.052951,0.016832
42974,0.000000,0.000000
42975,0.328885,0.164326


In [20]:
df_ht2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42977 entries, 0 to 42976
Data columns (total 85 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   anxiety          42977 non-null  int64  
 1   anxious          42977 non-null  int64  
 2   aroused          42977 non-null  int64  
 3   arthritis        42977 non-null  int64  
 4   creative         42977 non-null  int64  
 5   depression       42977 non-null  int64  
 6   dizzy            42977 non-null  int64  
 7   dry eyes         42977 non-null  int64  
 8   dry mouth        42977 non-null  int64  
 9   energetic        42977 non-null  int64  
 10  epilepsy         42977 non-null  int64  
 11  euphoric         42977 non-null  int64  
 12  eye pressure     42977 non-null  int64  
 13  fatigue          42977 non-null  int64  
 14  focused          42977 non-null  int64  
 15  giggly           42977 non-null  int64  
 16  happy            42977 non-null  int64  
 17  headache    

In [21]:
df_ht2

,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,dry mouth,energetic,...,vanilla,violet,woody,species,X..Delta9-THC,X..CBD,X..D-Limonene,X..Beta-Myrcene,X..Beta-Pinene,X..Linalool
0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
2,0,0,1,0,1,0,1,0,1,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42973,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42974,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02


In [69]:
df_combined

,0,1,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,...,vanilla,violet,woody,species,X..Delta9-THC,X..CBD,X..D-Limonene,X..Beta-Myrcene,X..Beta-Pinene,X..Linalool
0,0.416890,0.271077,0,0,0,0,0,0,1,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
1,0.347548,-0.019636,0,0,1,0,0,0,0,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
2,0.382990,0.217662,0,0,1,0,1,0,1,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
3,0.352342,0.100435,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
4,0.390504,-0.094678,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,0.195096,-0.048770,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42973,0.000000,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42974,0.000000,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42975,0.422824,0.203298,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02


In [70]:
X_train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25785 entries, 24748 to 39901
Data columns (total 81 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   0             25785 non-null  float64
 1   1             25785 non-null  float64
 2   anxiety       25785 non-null  int64  
 3   anxious       25785 non-null  int64  
 4   aroused       25785 non-null  int64  
 5   arthritis     25785 non-null  int64  
 6   creative      25785 non-null  int64  
 7   depression    25785 non-null  int64  
 8   dizzy         25785 non-null  int64  
 9   dry eyes      25785 non-null  int64  
 10  dry mouth     25785 non-null  int64  
 11  energetic     25785 non-null  int64  
 12  epilepsy      25785 non-null  int64  
 13  euphoric      25785 non-null  int64  
 14  eye pressure  25785 non-null  int64  
 15  fatigue       25785 non-null  int64  
 16  focused       25785 non-null  int64  
 17  giggly        25785 non-null  int64  
 18  happy         25785 no

In [71]:
X_train1

,0,1,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,...,sweet,tar,tea,tobacco,tree,tropical,vanilla,violet,woody,species
24748,0.000000,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4165,0.268846,-0.295688,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21594,0.287768,0.051506,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31547,0.404857,0.244676,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
38741,0.000000,0.000000,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469,0.311386,-0.206240,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
19121,0.334488,0.477166,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30140,0.446222,-0.257310,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
36028,0.427271,-0.152895,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [72]:
X_train1[X_train1[0].isnull()]

,0,1,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,...,sweet,tar,tea,tobacco,tree,tropical,vanilla,violet,woody,species


In [22]:
df_ht

,review,anxiety,anxious,aroused,arthritis,creative,depression,dizzy,dry eyes,dry mouth,...,vanilla,violet,woody,species,X..Delta9-THC,X..CBD,X..D-Limonene,X..Beta-Myrcene,X..Beta-Pinene,X..Linalool
0,migrain pain nausea medic round take bout min ...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
1,tast delici great overal well-balanc effect hi...,0,0,1,0,0,0,0,0,0,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
2,version outdoor grow loos bud good amount pain...,0,0,1,0,1,0,1,0,1,...,1,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
3,compar well hybrid like super lemon haze possi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
4,favorit strain tri low smell small pine start ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.5,0.01,0.27,0.62,0.15,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42972,love midnight daytim kind brain massag sweet s...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42973,knowledg quot blackberri domin cross black ras...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42974,chase,0,0,0,0,0,0,0,0,1,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02
42975,smoke felt good whole high get horni make happ...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,18.4,0.01,0.19,0.35,0.03,0.02


In [24]:
mae_train = []
mae_val = []
#for item in df_param.to_dict(orient='records')[:5]:
for item in df_param.to_dict(orient='records'):
    print(item)

    # input tfidf 
    tfidf = TfidfVectorizer(**item)
    tfidf_array = tfidf.fit_transform(df_ht['review'])
    lsa = TruncatedSVD()
    lsa_comps = lsa.fit_transform(tfidf_array)
    df_lsa = pd.DataFrame(lsa_comps)
    # combine 
    df_combined = pd.concat([df_lsa, df_no_tfidf], axis = 1)
    X1 = df_combined.drop(['X..Delta9-THC',
 'X..CBD',
 'X..CBN',
 'X..CBG',
 'X..CBC',
 'X..THCV',
 'X..D-Limonene',
 'X..Beta-Ocimene',
 'X..Beta-Myrcene',
 'X..Beta-Pinene',
 'X..Linalool',
 'X..Alpha-Pinene',
 'X..Camphene',
 'X..3-Carene',
 'X..Alpha-Terpinene',
 'X..Ocimene',
 'X..Eucalyptol',
 'X..Terpinolene',
 'X..Isopulegol',
 'X..Geraniol',
 'X..Guaiol',
 'X..CaryophylleneOxide',
 'X..Alpha-Bisabolol'], axis = 1)
    y1 = df_combined[['X..Delta9-THC',
 'X..CBD',
 'X..CBN',
 'X..CBG',
 'X..CBC',
 'X..THCV',
 'X..D-Limonene',
 'X..Beta-Ocimene',
 'X..Beta-Myrcene',
 'X..Beta-Pinene',
 'X..Linalool',
 'X..Alpha-Pinene',
 'X..Camphene',
 'X..3-Carene',
 'X..Alpha-Terpinene',
 'X..Ocimene',
 'X..Eucalyptol',
 'X..Terpinolene',
 'X..Isopulegol',
 'X..Geraniol',
 'X..Guaiol',
 'X..CaryophylleneOxide',
 'X..Alpha-Bisabolol']]
    # train test 
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=1, test_size=0.2)
    X_train1, X_val, y_train1, y_val = train_test_split(X_train, y_train, random_state=1, test_size=0.25)
    # fit ml 
    knn = KNeighborsRegressor()
    knn.fit(X_train1, y_train1)
    # eval

    #train 
    predict_knn_train = knn.predict(X_train1)
    mae_train.append(mean_absolute_error(y_train1, predict_knn_train))  

    # eval 
    predict_knn_val = knn.predict(X_val)
    mae_val.append(mean_absolute_error(y_val, predict_knn_val))

{'ngram_range': (1, 1), 'min_df': 0.01, 'max_df': 0.9}
{'ngram_range': (1, 1), 'min_df': 0.01, 'max_df': 0.95}
{'ngram_range': (1, 1), 'min_df': 0.01, 'max_df': 1.0}
{'ngram_range': (1, 1), 'min_df': 0.05, 'max_df': 0.9}
{'ngram_range': (1, 1), 'min_df': 0.05, 'max_df': 0.95}
{'ngram_range': (1, 1), 'min_df': 0.05, 'max_df': 1.0}
{'ngram_range': (1, 1), 'min_df': 0.1, 'max_df': 0.9}
{'ngram_range': (1, 1), 'min_df': 0.1, 'max_df': 0.95}
{'ngram_range': (1, 1), 'min_df': 0.1, 'max_df': 1.0}
{'ngram_range': (1, 2), 'min_df': 0.01, 'max_df': 0.9}
{'ngram_range': (1, 2), 'min_df': 0.01, 'max_df': 0.95}
{'ngram_range': (1, 2), 'min_df': 0.01, 'max_df': 1.0}
{'ngram_range': (1, 2), 'min_df': 0.05, 'max_df': 0.9}
{'ngram_range': (1, 2), 'min_df': 0.05, 'max_df': 0.95}
{'ngram_range': (1, 2), 'min_df': 0.05, 'max_df': 1.0}
{'ngram_range': (1, 2), 'min_df': 0.1, 'max_df': 0.9}
{'ngram_range': (1, 2), 'min_df': 0.1, 'max_df': 0.95}
{'ngram_range': (1, 2), 'min_df': 0.1, 'max_df': 1.0}


In [120]:
#df_param_mini = df_param.head()

In [121]:
#df_param_mini['train_mae'] = mae_train
#df_param_mini['val_mae'] = mae_val

/var/folders/y1/pjvjlkjn5gl846rnyzr53p340000gn/T/ipykernel_1655/3355292209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_param_mini['train_mae'] = mae_train
/var/folders/y1/pjvjlkjn5gl846rnyzr53p340000gn/T/ipykernel_1655/3355292209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_param_mini['val_mae'] = mae_val


In [122]:
df_param_mini

,ngram_range,min_df,max_df,train_mae,val_mae
0,"(1, 1)",0.01,0.90,0.847127,1.059881
1,"(1, 1)",0.01,0.95,0.843991,1.060232
2,"(1, 1)",0.01,1.00,0.843560,1.061116
3,"(1, 1)",0.05,0.90,0.849682,1.064991
4,"(1, 1)",0.05,0.95,0.852870,1.067704


In [25]:
df_param

,ngram_range,min_df,max_df
0,"(1, 1)",0.01,0.90
1,"(1, 1)",0.01,0.95
2,"(1, 1)",0.01,1.00
3,"(1, 1)",0.05,0.90
4,"(1, 1)",0.05,0.95
5,"(1, 1)",0.05,1.00
6,"(1, 1)",0.10,0.90
7,"(1, 1)",0.10,0.95
8,"(1, 1)",0.10,1.00
9,"(1, 2)",0.01,0.90


In [26]:
df_param['train_mae'] = mae_train
df_param['val_mae'] = mae_val

In [27]:
df_param

,ngram_range,min_df,max_df,train_mae,val_mae
0,"(1, 1)",0.01,0.90,0.069580,0.088428
1,"(1, 1)",0.01,0.95,0.069445,0.088363
2,"(1, 1)",0.01,1.00,0.069543,0.088302
3,"(1, 1)",0.05,0.90,0.069988,0.088208
4,"(1, 1)",0.05,0.95,0.069911,0.088201
5,"(1, 1)",0.05,1.00,0.069832,0.088071
6,"(1, 1)",0.10,0.90,0.070885,0.088573
7,"(1, 1)",0.10,0.95,0.071024,0.088557
8,"(1, 1)",0.10,1.00,0.070936,0.088732
9,"(1, 2)",0.01,0.90,0.069332,0.088335


In [28]:
import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [29]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andalanputra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andalanputra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andalanputra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/andalanputra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/andalanputra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
      res=[]
      for token in word_tokenize(doc):
        word=''.join(filter(str.isalnum, token))
        if ((word not in self.ignore_tokens) and (len(self.wnl.lemmatize(word))>2) and not(any(char.isdigit() for char in word))):
          res.append(self.wnl.lemmatize(word))
      return res

stop_words = nltk.corpus.stopwords.words('english')
#stop_words.extend(["more","words"])

tokenizer = LemmaTokenizer()
token_stop = list(set(tokenizer(' '.join(stop_words))))
token_stop.sort()

In [31]:
df_ht['review']

0        migrain pain nausea medic round take bout min ...
1        tast delici great overal well-balanc effect hi...
2        version outdoor grow loos bud good amount pain...
3        compar well hybrid like super lemon haze possi...
4        favorit strain tri low smell small pine start ...
                               ...                        
42972    love midnight daytim kind brain massag sweet s...
42973    knowledg quot blackberri domin cross black ras...
42974                                                chase
42975    smoke felt good whole high get horni make happ...
42976    make overal good make cum load tran gay bisexu...
Name: review, Length: 42977, dtype: object

In [32]:
list_of_lists = [s.split() for s in df_ht['review'].tolist()]

In [33]:
list_of_lists

[['migrain',
  'pain',
  'nausea',
  'medic',
  'round',
  'take',
  'bout',
  'min',
  'start',
  'work',
  'absolut',
  'must',
  'head',
  'high',
  'kinda',
  'heavi',
  'couch',
  'lock',
  'nice',
  'bodi',
  'high',
  'exactli',
  'need',
  'one',
  'good',
  'use',
  'definit',
  'good',
  'bedtim',
  'highli',
  'recommend',
  'even',
  'use'],
 ['tast',
  'delici',
  'great',
  'overal',
  'well-balanc',
  'effect',
  'high',
  'right',
  'daytim',
  'use'],
 ['version',
  'outdoor',
  'grow',
  'loos',
  'bud',
  'good',
  'amount',
  'pain',
  'relief',
  'pretti',
  'spacey',
  'high',
  'lose',
  'track',
  'time',
  'focu',
  'random',
  'thing',
  'definit',
  'increas',
  'appetit',
  'thirst',
  'well',
  'feel',
  'mildli',
  'psychedel',
  'alter',
  'percept',
  'surround',
  'sometim',
  'accompani',
  'mild',
  'dizzi'],
 ['compar',
  'well',
  'hybrid',
  'like',
  'super',
  'lemon',
  'haze',
  'possibl',
  'even',
  'blue',
  'dream',
  'like',
  'content',
 

In [34]:
tfidf_vectorizer = TfidfVectorizer(stop_words=token_stop, preprocessor=' '.join, lowercase=True, strip_accents='unicode', min_df=0.01, max_df=1.00, ngram_range=(1,1))
tfidf_array_ht = tfidf_vectorizer.fit_transform(list_of_lists)
feature_names = tfidf_vectorizer.get_feature_names()
print(feature_names)

['abl', 'absolut', 'activ', 'actual', 'almost', 'also', 'alway', 'amaz', 'amount', 'amp', 'anoth', 'anxieti', 'anxiou', 'anyon', 'anyth', 'appetit', 'aroma', 'around', 'asleep', 'away', 'awesom', 'back', 'bad', 'bake', 'balanc', 'batch', 'beauti', 'becom', 'bed', 'berri', 'best', 'better', 'big', 'bit', 'blue', 'blueberri', 'bodi', 'bong', 'bowl', 'bring', 'bud', 'burn', 'buy', 'buzz', 'call', 'calm', 'cannabi', 'care', 'caus', 'cerebr', 'cherri', 'chill', 'chronic', 'citru', 'clean', 'clear', 'color', 'come', 'complet', 'concentr', 'couch', 'cough', 'coupl', 'cover', 'creativ', 'crystal', 'dank', 'dark', 'day', 'daytim', 'decent', 'deep', 'definit', 'delici', 'dens', 'depress', 'diesel', 'differ', 'disappoint', 'dispensari', 'domin', 'dream', 'dri', 'earthi', 'easi', 'eat', 'effect', 'end', 'energ', 'energet', 'energi', 'enjoy', 'enough', 'especi', 'euphor', 'euphoria', 'even', 'ever', 'everi', 'everyth', 'excel', 'exhal', 'expect', 'experi', 'extrem', 'eye', 'face', 'fall', 'fan', 'f

In [35]:
len(feature_names)

347

In [36]:
n_components = [2, 173, 346]
param_list2 = []
for d in n_components:
    param_list2.append([d])

In [37]:
df_param2 = pd.DataFrame(param_list2, columns = ['n_components'])

In [38]:
mae_train2 = []
mae_val2 = []
#for item in df_param.to_dict(orient='records')[:5]:
for item2 in df_param2.to_dict(orient='records'):
    print(item2)

    # input tfidf 
    #tfidf = TfidfVectorizer(**item)
    #tfidf_array = tfidf.fit_transform(df_ht['review'])

    #input svd
    lsa2 = TruncatedSVD(**item2)
    lsa_comps2 = lsa2.fit_transform(tfidf_array_ht)
    df_lsa2 = pd.DataFrame(lsa_comps2)
    # combine 
    df_combined2 = pd.concat([df_lsa2, df_no_tfidf], axis = 1)
    X2 = df_combined2.drop(['X..Delta9-THC',
 'X..CBD',
 'X..CBN',
 'X..CBG',
 'X..CBC',
 'X..THCV',
 'X..D-Limonene',
 'X..Beta-Ocimene',
 'X..Beta-Myrcene',
 'X..Beta-Pinene',
 'X..Linalool',
 'X..Alpha-Pinene',
 'X..Camphene',
 'X..3-Carene',
 'X..Alpha-Terpinene',
 'X..Ocimene',
 'X..Eucalyptol',
 'X..Terpinolene',
 'X..Isopulegol',
 'X..Geraniol',
 'X..Guaiol',
 'X..CaryophylleneOxide',
 'X..Alpha-Bisabolol'], axis = 1)
    y2 = df_combined2[['X..Delta9-THC',
 'X..CBD',
 'X..CBN',
 'X..CBG',
 'X..CBC',
 'X..THCV',
 'X..D-Limonene',
 'X..Beta-Ocimene',
 'X..Beta-Myrcene',
 'X..Beta-Pinene',
 'X..Linalool',
 'X..Alpha-Pinene',
 'X..Camphene',
 'X..3-Carene',
 'X..Alpha-Terpinene',
 'X..Ocimene',
 'X..Eucalyptol',
 'X..Terpinolene',
 'X..Isopulegol',
 'X..Geraniol',
 'X..Guaiol',
 'X..CaryophylleneOxide',
 'X..Alpha-Bisabolol']]
    # train test 
    #X_train_, X_test_, y_train_, y_test_ = train_test_split(df_combined2.drop(["X..Delta9-THC", "X..CBD", "X..D-Limonene", "X..Beta-Myrcene", "X..Beta-Pinene", "X..Linalool"], axis = 1), df_combined2[["X..Delta9-THC", "X..CBD", "X..D-Limonene", "X..Beta-Myrcene", "X..Beta-Pinene", "X..Linalool"]], random_state=1, test_size=0.2)
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X2, y2, random_state=1, test_size=0.2)
    X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train_, y_train_, random_state=1, test_size=0.25)
    # fit ml 
    knn2 = KNeighborsRegressor()
    knn2.fit(X_train2, y_train2)
    # eval

    #train 
    predict_knn_train2 = knn2.predict(X_train2)
    mae_train2.append(mean_absolute_error(y_train2, predict_knn_train2))  

    # eval 
    predict_knn_val2 = knn2.predict(X_val2)
    mae_val2.append(mean_absolute_error(y_val2, predict_knn_val2))

{'n_components': 2}
{'n_components': 173}
{'n_components': 346}


In [39]:
df_param2

,n_components
0,2
1,173
2,346


In [40]:
df_param2['train_mae_svd'] = mae_train2
df_param2['val_mae_svd'] = mae_val2

In [41]:
df_param2

,n_components,train_mae_svd,val_mae_svd
0,2,0.069467,0.088240
1,173,0.069251,0.087986
2,346,0.069227,0.088045


word cloud (after truncated svd)

In [50]:
def topics_top_words(lsa, feature_names, n_top_words=15):
  top_words = []
  for topic in lsa.components_:
        top_words.append(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))
  return top_words

lsa_topics = topics_top_words(lsa, feature_names)

In [51]:
lsa_topics

['away amount alway asleep also awesom almost amp actual aroma anyon around anxiou anxieti anyth',
 'away anoth anyon activ back anyth anxieti almost appetit alway absolut asleep actual aroma amaz']

In [ ]:
# using WordCloud for:
# WordCloud is obtained from SVD, each text representig as a vector (Document space), giving you what is happening within the word spases(wirh different values and dimensions)
# trying to figure out the most importnat words in each one of the dimensions (so you have an idea what is happening in the dimensions/topics)
# wordcloud gives you the most importnat words in the dimensions

# size prob representing how importnat the words are
# ex. "manager" =  people are talking for it in a positive manner, people were happy about it

#from wordcloud import WordCloud

# Generate a word cloud image
wordcloud = WordCloud().generate(positive_topics[0])

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(positive_topics[1])
plt.figure(figsize=(24,12))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

word cloud (without truncated svd)